# Data Cleaning Listado de Listado de Beneficiarios 2021

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. beneficiarios_21 -> Dataset con el listado de TODOS los beneficiarios de 2021.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura y limpieza de datos

### 1.1 Beneficiarios 2021

Obtenemos el listado único de localidades y listado único de municipios con sus claves.

In [3]:
beneficiarios_21 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/listado_beneficiarios_fertilizantes_2021.csv', encoding='utf-8', skiprows=0)

In [4]:
beneficiarios_21.shape

(394986, 16)

In [5]:
beneficiarios_21 = beneficiarios_21[~((beneficiarios_21['ENTIDAD'] == 'NACIONAL') & (beneficiarios_21['MUNICIPIO'] == 'NACIONAL') & (beneficiarios_21['LOCALIDAD'] == 'NACIONAL'))]

In [6]:
beneficiarios_21['ENTIDAD'].unique()

array(['GUERRERO', 'PUEBLA', 'MORELOS', 'TLAXCALA', nan], dtype=object)

Dropeamos los tres valores nulos debido a que las rows enteras salen como nan

In [8]:
beneficiarios_21.dropna(inplace=True)

In [10]:
# Obtenemos las localidades únicas en el dataset.
Localidades_21 = beneficiarios_21[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_21 = Localidades_21.drop_duplicates()


In [11]:
Localidades_21['ENTIDAD_c_benef'] = Localidades_21['ENTIDAD'].apply(clean_text)
Localidades_21['MUNICIPIO_c_benef'] = Localidades_21['MUNICIPIO'].apply(clean_text)
Localidades_21['LOCALIDAD_c_benef'] = Localidades_21['LOCALIDAD'].apply(clean_text)

In [12]:
# Creamos las dos keys de beneficiarios 2019
Localidades_21['KEY_benef_mun'] = Localidades_21['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_21[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_21['KEY_benef_loc'] = Localidades_21['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_21[
    'MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_21['LOCALIDAD_c_benef'].astype(str)

In [13]:
Localidades_21.shape

(5914, 8)

In [14]:
# Obtenemos las localidades únicas en el dataset.
Municipios_21 = beneficiarios_21[['ENTIDAD', 'MUNICIPIO']]
Municipios_21 = Municipios_21.drop_duplicates()

In [15]:
# Estandarizamos la limpieza de los datos
Municipios_21['ENTIDAD_c_benef'] = Municipios_21['ENTIDAD'].apply(clean_text)
Municipios_21['MUNICIPIO_c_benef'] = Municipios_21['MUNICIPIO'].apply(clean_text)

In [16]:
# Creamos las dos keys de beneficiarios 
Municipios_21['KEY_benef_mun'] = Municipios_21['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_21[
    'MUNICIPIO_c_benef'].astype(str)

In [17]:
Municipios_21.shape

(248, 5)

In [18]:
Municipios_21.columns

Index(['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef_mun'],
      dtype='object')

### 1.2 INEGI 2021

Obtener listado único de municipios y listado único de localidades de inegi de 2021 con sus claves.

In [19]:
path_dataset_inegi_2021 = '../../data/inegi/dataset_inegi_clean_2021.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2021 = pd.read_csv(path_dataset_inegi_2021)

### 1.2.1 INEGI 2021 Municipios únicos para cada año.

In [20]:
dataset_inegi_2021['KEY_inegi_municipio'] = dataset_inegi_2021['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2021[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2021['KEY_inegi_localidad'] = dataset_inegi_2021['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2021[
    'Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2021['Localidad_c_inegi'].astype(str)

In [21]:
INEGI_UNIQUEMUN_2021 = dataset_inegi_2021.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad"])

INEGI_UNIQUEMUN_2021 = INEGI_UNIQUEMUN_2021.drop_duplicates()

In [22]:
INEGI_UNIQUEMUN_2021.shape

(2471, 7)

In [23]:
INEGI_UNIQUEMUN_2021.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi_municipio'],
      dtype='object')

### 1.2.1 INEGI 2021 Localidades únicas para cada año.

In [24]:
INEGI_UNIQUELOC_2021 = dataset_inegi_2021

INEGI_UNIQUELOC_2021 = INEGI_UNIQUELOC_2021.drop_duplicates()

In [25]:
INEGI_UNIQUELOC_2021.shape

(300350, 11)

# 2. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2019

El objetivo de esta sección es crear dos diccionarios de códigos según BENEFICIARIOS-MUNICIPIOS_INEGI y otro BENEFICIARIOS-LOCALIDADES_INEGI, para cada uno de los estados encontrados en inegi_2021, en este caso el número de keys es menor con lo cual no nos hará falta dividir por estado.

Para ello haremos un Left join entre Localidades_21 y el dataset de INEGI correspondiente.

In [26]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 85 y 100 de coincidencia
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=85, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()
    
    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))


    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]
    
    df_benef['match_score'] = [match[1] if match else None for match in matches]
    

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on='best_match', right_on=key2, how='left',
                         suffixes=('_benef', '_inegi'))
    
    return df_merged

In [27]:
diccionario_MUN_21 = fuzzy_merge_benef2019_2022(Municipios_21, INEGI_UNIQUEMUN_2021, 'KEY_benef_mun', 'KEY_inegi_municipio')
diccionario_MUN_21.shape

(248, 14)

In [28]:
diccionario_MUN_21.drop(columns=['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef', 'Entidad_c_inegi',
       'Municipio_c_inegi'], inplace=True)

In [29]:
diccionario_MUN_21.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_21.csv', index=False)

In [30]:
diccionario_MUN_21_simple = pd.read_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_21_simple.csv')

In [31]:
nan_rows = beneficiarios_21[beneficiarios_21.isna().any(axis=1)]

In [32]:
nan_rows

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA


In [33]:
# Assuming your DataFrame is named df
filtered_df = beneficiarios_21[beneficiarios_21['ENTIDAD'] == 'NACIONAL']

In [34]:
filtered_df

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA


### MERGE

Armamos por partes el dataset definitivo:
1. Juntamos beneficiarios_19 con el diccionario simple. (listado_beneficiario_parte_I)
2. Al df anterior juntamos las claver provenientes del catálogo de Inegi. (listado_beneficiario_parte_II)

In [35]:
beneficiarios_21.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA'],
      dtype='object')

In [36]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join

# Clean listado beneficiarios
beneficiarios_21['ESTADO_Clean'] = beneficiarios_21['ENTIDAD'].apply(clean_text)
beneficiarios_21['MUNICIPIO_Clean'] = beneficiarios_21['MUNICIPIO'].apply(clean_text)

# Create KEY in listado beneficiarios
beneficiarios_21['Estado-mun-KEY'] = beneficiarios_21['ESTADO_Clean'].astype(str) + '-' + beneficiarios_21[
    'MUNICIPIO_Clean'].astype(str)

In [37]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I = pd.merge(beneficiarios_21, diccionario_MUN_21_simple, left_on="Estado-mun-KEY",
                                        right_on="KEY_benef_mun", how='left', suffixes=('_benef', '_inegi'))

In [38]:
listado_beneficiarios_parte_I.isna().sum()

BENEFICIARIO           0
ZONA                   0
ENTIDAD                0
MUNICIPIO              0
LOCALIDAD              0
ESTRATIFICACIÓN        0
PROGRAMA               0
COMPONENTE             0
SUBCOMPONENTE          0
PRODUCTO               0
FECHA                  0
MONTO FEDERAL          0
APOYO                  0
ACTIVIDAD              0
ESLABÓN                0
CICLO AGRÍCOLA         0
ESTADO_Clean           0
MUNICIPIO_Clean        0
Estado-mun-KEY         0
KEY_benef_mun          0
KEY_inegi_municipio    0
dtype: int64

In [39]:
listado_beneficiarios_parte_I.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio'],
      dtype='object')

In [40]:
# Hacer el join de la Parte II
listado_beneficiarios_parte_II = pd.merge(listado_beneficiarios_parte_I, INEGI_UNIQUEMUN_2021, left_on="KEY_inegi_municipio",
                                        right_on="KEY_inegi_municipio", how='left', suffixes=('_benef', '_inegi'))

In [41]:
listado_beneficiarios_parte_II

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,...,MUNICIPIO_Clean,Estado-mun-KEY,KEY_benef_mun,KEY_inegi_municipio,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,Entidad_c_inegi,Municipio_c_inegi
0,ABAC NERI JESUS CARLOS,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,12,Guerrero,21,Coyuca de Benítez,guerrero,coyuca de benitez
1,ABAD ALARCON ELIAZAR,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,12,Guerrero,21,Coyuca de Benítez,guerrero,coyuca de benitez
2,ABAD ALARCON EZEQUIEL,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,12,Guerrero,21,Coyuca de Benítez,guerrero,coyuca de benitez
3,ABAD ALARCON FULGENCIO,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,12,Guerrero,21,Coyuca de Benítez,guerrero,coyuca de benitez
4,ABAD ALARCON ROMALDA,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,YERBASANTITA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,12,Guerrero,21,Coyuca de Benítez,guerrero,coyuca de benitez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394978,ZURITA VILLEGAS CRESCENCIANA,CENTRO PAÍS,GUERRERO,XALPATLÁHUAC,XALPATLÁHUAC,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,xalpatlahuac,guerrero-xalpatlahuac,guerrero-xalpatlahuac,guerrero-xalpatlahuac,12,Guerrero,69,Xalpatláhuac,guerrero,xalpatlahuac
394979,ZURITA VILLEGAS MARIA INES,CENTRO PAÍS,GUERRERO,XALPATLÁHUAC,XALPATLÁHUAC,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,xalpatlahuac,guerrero-xalpatlahuac,guerrero-xalpatlahuac,guerrero-xalpatlahuac,12,Guerrero,69,Xalpatláhuac,guerrero,xalpatlahuac
394980,ZURITA VILLEGAS NICOLASA,CENTRO PAÍS,GUERRERO,XALPATLÁHUAC,XALPATLÁHUAC,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,xalpatlahuac,guerrero-xalpatlahuac,guerrero-xalpatlahuac,guerrero-xalpatlahuac,12,Guerrero,69,Xalpatláhuac,guerrero,xalpatlahuac
394981,ZURITA VITINIO GREGORIA,CENTRO PAÍS,GUERRERO,XALPATLÁHUAC,XALPATLÁHUAC,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,xalpatlahuac,guerrero-xalpatlahuac,guerrero-xalpatlahuac,guerrero-xalpatlahuac,12,Guerrero,69,Xalpatláhuac,guerrero,xalpatlahuac


In [42]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio', 'CVE_ENT', 'Entidad_inegi',
       'CVE_MUN', 'Municipio_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi'],
      dtype='object')

In [43]:
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II.drop(columns=['ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY', 'Entidad_c_inegi','Municipio_c_inegi', 'KEY_inegi_municipio', 'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'APOYO', 'ACTIVIDAD', 'ESLABÓN'])

In [44]:
listado_beneficiarios_parte_II

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,KEY_benef_mun,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi
0,ABAC NERI JESUS CARLOS,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Cultivo de maíz grano blanco,2021-04-10,7196.13,PV2021,guerrero-coyuca de benitez,12,Guerrero,21,Coyuca de Benítez
1,ABAD ALARCON ELIAZAR,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Cultivo de maíz grano blanco,2021-04-10,3598.07,PV2021,guerrero-coyuca de benitez,12,Guerrero,21,Coyuca de Benítez
2,ABAD ALARCON EZEQUIEL,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Cultivo de maíz grano blanco,2021-04-06,7196.13,PV2021,guerrero-coyuca de benitez,12,Guerrero,21,Coyuca de Benítez
3,ABAD ALARCON FULGENCIO,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Cultivo de maíz grano blanco,2021-04-06,7196.13,PV2021,guerrero-coyuca de benitez,12,Guerrero,21,Coyuca de Benítez
4,ABAD ALARCON ROMALDA,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,YERBASANTITA,Cultivo de maíz grano blanco,2021-04-04,7196.13,PV2021,guerrero-coyuca de benitez,12,Guerrero,21,Coyuca de Benítez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394978,ZURITA VILLEGAS CRESCENCIANA,CENTRO PAÍS,GUERRERO,XALPATLÁHUAC,XALPATLÁHUAC,Cultivo de maíz grano blanco,2021-05-08,3598.07,PV2021,guerrero-xalpatlahuac,12,Guerrero,69,Xalpatláhuac
394979,ZURITA VILLEGAS MARIA INES,CENTRO PAÍS,GUERRERO,XALPATLÁHUAC,XALPATLÁHUAC,Cultivo de maíz grano blanco,2021-05-08,3598.07,PV2021,guerrero-xalpatlahuac,12,Guerrero,69,Xalpatláhuac
394980,ZURITA VILLEGAS NICOLASA,CENTRO PAÍS,GUERRERO,XALPATLÁHUAC,XALPATLÁHUAC,Cultivo de maíz grano blanco,2021-05-08,3598.07,PV2021,guerrero-xalpatlahuac,12,Guerrero,69,Xalpatláhuac
394981,ZURITA VITINIO GREGORIA,CENTRO PAÍS,GUERRERO,XALPATLÁHUAC,XALPATLÁHUAC,Cultivo de maíz grano blanco,2021-05-08,3598.07,PV2021,guerrero-xalpatlahuac,12,Guerrero,69,Xalpatláhuac


In [45]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi'],
      dtype='object')

In [46]:
listado_beneficiarios_parte_II.to_csv('../../data/listados_completos/listado_beneficiarios_2021.csv', index=False)

In [47]:
listado_beneficiarios_parte_II.isna().sum()

BENEFICIARIO       0
ZONA               0
ENTIDAD            0
MUNICIPIO          0
LOCALIDAD          0
PRODUCTO           0
FECHA              0
MONTO FEDERAL      0
CICLO AGRÍCOLA     0
KEY_benef_mun      0
CVE_ENT            0
Entidad_inegi      0
CVE_MUN            0
Municipio_inegi    0
dtype: int64

In [48]:
# Assuming listado_beneficiarios_parte_II is your DataFrame
rows_with_nan = listado_beneficiarios_parte_II[listado_beneficiarios_parte_II.isna().any(axis=1)]

# rows_with_nan now contains only the rows with NaN values in any column

In [49]:
rows_with_nan

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,KEY_benef_mun,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi


In [50]:
listado_beneficiarios_parte_II.shape

(394983, 14)